In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-26 06:10:03.843375: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 06:10:03.874137: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
#source_data = source_data.train_test_split(train_size=0.2, test_size=0.08)
#source_data

In [4]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [11]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [12]:
import adapters.composition as ac

#domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain_adapter_telephone_travel",with_head=False)
mlm_adapter = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/standalone-mlm-target",with_head=False)
task_adapter = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/task_adapter",with_head=False)


In [13]:
model.add_adapter_fusion(ac.Fuse(mlm_adapter, task_adapter))
model.add_classification_head("mnli", num_labels=3)
adapter_setup_fuse = ac.Fuse( mlm_adapter, task_adapter)
model.train_adapter_fusion(adapter_setup_fuse)

#model.active_adapters = ac.Stack(domain_adapter_coral, ac.Fuse(mlm_adapter, task_adapter))
model.active_adapters = ac.Fuse(mlm_adapter, task_adapter)



In [14]:
fn.print_trainable_parameters(model)

trainable params: 11841597 || all params: 82197597 || trainable%: 14.406256912838948


In [15]:
model.adapter_summary(as_dict=True)

[{'name': 'standalone-mlm-target',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3545856,
  'train': False,
  '%param': 4.605670051672885},
 {'name': 'task_adapter',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': False,
  '%param': 0.5809458731520459},
 {'name': 'Full model', '#param': 76988928, '%param': 100.0, 'train': False}]

In [16]:
model

DistilBertAdapterModel(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlockWithAdapters(
          (attention): MultiHeadSelfAttentionWithAdapters(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): LoRALinear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (k_lin): LoRALinear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (v_lin): LoRALinear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (out_lin): Linear(

In [17]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
print(model.active_adapters)

distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.standalone-mlm-target,task_adapter.query.weight
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.standalone-mlm-target,task_adapter.query.bias
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.standalone-mlm-target,task_adapter.key.weight
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.standalone-mlm-target,task_adapter.key.bias
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.standalone-mlm-target,task_adapter.value.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.standalone-mlm-target,task_adapter.query.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.standalone-mlm-target,task_adapter.query.bias
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.standalone-mlm-target,task_adapter.key.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.standalone-mlm-t

In [18]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.2965587044534413
F1 score before adaptation: 0.187969478812992


In [19]:

reload(fn)
trainer = fn.train_model(model,"multi_block-without-coral",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

{'loss': 0.7598, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.8415830135345459, 'eval_accuracy': 0.6224951519069166, 'eval_f1': 0.616157178223111, 'eval_precision': 0.7213893782876665, 'eval_recall': 0.6224951519069166, 'eval_runtime': 16.0044, 'eval_samples_per_second': 483.306, 'eval_steps_per_second': 15.121, 'epoch': 0.21}
{'loss': 0.6958, 'learning_rate': 9.436936936936938e-05, 'epoch': 0.43}


...

{'eval_loss': 0.6911162734031677, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.7055692807855044, 'eval_precision': 0.7230116216665397, 'eval_recall': 0.7058823529411765, 'eval_runtime': 16.1503, 'eval_samples_per_second': 478.938, 'eval_steps_per_second': 14.984, 'epoch': 3.62}
{'loss': 0.6416, 'learning_rate': 4.279279279279279e-06, 'epoch': 3.84}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.6857745051383972, 'eval_accuracy': 0.7097608274078863, 'eval_f1': 0.7087684757794204, 'eval_precision': 0.7234510751705419, 'eval_recall': 0.7097608274078863, 'eval_runtime': 16.1585, 'eval_samples_per_second': 478.694, 'eval_steps_per_second': 14.977, 'epoch': 3.84}


Overwriting existing adapter 'standalone-mlm-target'.
Overwriting existing adapter 'task_adapter'.
Overwriting existing adapter fusion module 'standalone-mlm-target,task_adapter'


{'train_runtime': 1634.5757, 'train_samples_per_second': 183.566, 'train_steps_per_second': 5.738, 'train_loss': 0.6693706764595341, 'epoch': 4.0}


In [19]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.6938259109311741
F1 score before adaptation: 0.6944925204457613


In [17]:
# Evaluate the model on the target domain before adaptation
#this is using the full sourcec domain data
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7211538461538461
F1 score before adaptation: 0.7214558115652716


In [20]:
# Evaluate the model on the target domain before adaptation
#final
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7186234817813765
F1 score before adaptation: 0.7189379678534682
